In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import MultiComparison
from sklearn.cluster import KMeans
import numpy as np
pd.set_option('display.width', 2000)

In [ ]:
nucleus_df = pd.read_csv('csv/nuclei_iba1.csv')
image_df = pd.read_csv('csv/images_iba1.csv')
print(nucleus_df.keys())

In [ ]:
missing_ch4 = nucleus_df['Ch4Intensity'].isnull()

# Shift the values for rows with missing Ch4Intensity
nucleus_df.loc[missing_ch4, 'Ch4Intensity'] = nucleus_df.loc[missing_ch4, 'Ch3Intensity']
nucleus_df.loc[missing_ch4, 'Ch3Intensity'] = nucleus_df.loc[missing_ch4, 'Ch2Intensity']
nucleus_df.loc[missing_ch4, 'Ch2Intensity'] = np.nan
nucleus_df.loc[missing_ch4, 'CytoCh4Intensity'] = nucleus_df.loc[missing_ch4, 'CytoCh3Intensity']
nucleus_df.loc[missing_ch4, 'CytoCh3Intensity'] = nucleus_df.loc[missing_ch4, 'CytoCh2Intensity']
nucleus_df.loc[missing_ch4, 'CytoCh2Intensity'] = np.nan

In [ ]:

#nucleus_df = nucleus_df[nucleus_df['Location'] == 'DG']
mean_intensity = nucleus_df.groupby(['Condition', 'Location', 'ImageName'])['CytoCh1Intensity'].mean().reset_index()
print(mean_intensity)
#mean_intensity = mean_intensity[mean_intensity['Location']=='HC']
#grouped_df=merged_df.groupby('ImageName')

order = ['Sham', 'Contra', 'Ipsi']

# Plotting
sns.boxplot(data=mean_intensity, x='Condition', y='CytoCh1Intensity', hue = 'Location',order=order, palette='Set3')
plt.ylabel('Mean cytoplasmic GFAP intensity')
plt.show()

In [ ]:
import napari
all_sorted_clusters = []
channel_key = 'CytoCh1Intensity'
nucleus_df['Cluster_Label'] = np.nan
num_clusters = 2

for name in nucleus_df['ImageName'].unique():
    df = nucleus_df[nucleus_df['ImageName'] == name]
    intensity_values = df[channel_key].values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init="auto")
    clusters = kmeans.fit_predict(intensity_values)
    sorted_clusters = np.argsort(kmeans.cluster_centers_.flatten())
    cluster_labels = sorted_clusters[clusters] 
    nucleus_df.loc[df.index, 'Cluster_Label'] = cluster_labels

nucleus_df.loc[nucleus_df['Cluster_Label'] == 1.0, 'CellType'] = 'Astrocyte'
nucleus_df.drop(columns=['Cluster_Label'], inplace=True)
print(nucleus_df.head())

astrocyte_df = nucleus_df[nucleus_df['CellType'] == 'Astrocyte']


# grouped_df = positive_df.groupby(['ImageName', 'Condition', 'Location']).size().unstack(fill_value=0)
# #print(grouped_df)
# merged_df = pd.merge(grouped_df, image_df[['ImageName','Condition', 'CA1Volume']], on='ImageName')

# # Reset index to make 'ImageName', 'Condition', and 'Cluster_Label' as columns
# scale = [0.9278, 0.3459, 0.3459]
# grouped_df.reset_index(inplace=True)
# merged_df['positiveVolume'] = (merged_df['HC'] / merged_df['CA1Volume']) * np.prod(scale) *10**6
# print(positive_df)


In [ ]:
all_sorted_clusters = []
channel_key = 'CytoCh3Intensity'
nucleus_df['Cluster_Label'] = np.nan
num_clusters = 2

for name in nucleus_df['ImageName'].unique():
    df = nucleus_df[nucleus_df['ImageName'] == name]
    intensity_values = df[channel_key].values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init="auto")
    clusters = kmeans.fit_predict(intensity_values)
    sorted_clusters = np.argsort(kmeans.cluster_centers_.flatten())
    cluster_labels = sorted_clusters[clusters] 
    nucleus_df.loc[df.index, 'Cluster_Label'] = cluster_labels

nucleus_df.loc[(nucleus_df['Cluster_Label'] == 1.0) & (nucleus_df['CellType'] != 'Astrocyte'), 'CellType'] = 'Microglia'
nucleus_df.drop(columns=['Cluster_Label'], inplace=True)
microglia_df = nucleus_df[nucleus_df['CellType'] == 'Microglia']
print(nucleus_df.head())

In [ ]:
negative_df = nucleus_df[nucleus_df['CellType'] != 'Astrocyte']

neg_mean_intensity = negative_df.groupby(['Condition', 'Location', 'ImageName'])['CytoCh1Intensity'].mean().reset_index()
print(neg_mean_intensity)
#neg_mean_intensity = neg_mean_intensity[neg_mean_intensity['Location']=='EDGE']
#grouped_df=merged_df.groupby('ImageName')

order = ['Sham', 'Contra', 'Ipsi']
palette = sns.color_palette(['royalblue'], 2)
# Plotting
sns.boxplot(data=neg_mean_intensity, x='Condition', y='CytoCh1Intensity', hue='Location', order=order, palette='Set3')
sns.stripplot(data=neg_mean_intensity, x='Condition', y='CytoCh1Intensity', order=order, hue='Location', palette=palette, alpha=0.7,dodge=True)
plt.ylabel('Mean cytoplasmic GFAP intensity')
plt.tight_layout()
plt.savefig('non_astrocyte_GFAP.pdf')
plt.show()

In [ ]:

imgName = 'HI 1 Ipsilateral Mouse 8 Slide 22  IBA1green GFAPpink Col1a1red 40x 6x6 technical replica 2.lsm'
img1_astrocyte_df = astrocyte_df[astrocyte_df['ImageName'] == imgName]
img1_microglia_df = microglia_df[microglia_df['ImageName'] == imgName]

centroidsAstro = img1_astrocyte_df['Centroid'].apply(eval)
centroidsMicro = img1_microglia_df['Centroid'].apply(eval)

# Convert centroids to a NumPy array
centroids_array_astro = np.array(centroidsAstro.tolist())  # Transpose the array to get the desired shape
centroids_array_micro = np.array(centroidsMicro.tolist())

from skimage import io
img= io.imread("D:/Users\Jonas/nuclei\imagesAndMasks\iba1\ipsi\HI 1 Ipsilateral Mouse 8 Slide 22  IBA1green GFAPpink Col1a1red 40x 6x6 technical replica 2.lsm")

viewer = napari.view_image(img, scale =([0.9278, 0.3459, 0.3459]), channel_axis=3 )
viewer.add_points(centroids_array_astro, size=10, symbol='cross', edge_color='blue', face_color='blue', name='Astrocytes', scale=([0.9278, 0.3459, 0.3459]))
viewer.add_points(centroids_array_micro, size=10, symbol='cross', edge_color='green', face_color='green', name='Microglia', scale=([0.9278, 0.3459, 0.3459]))
napari.run()

In [ ]:
grouped_df = astrocyte_df.groupby(['ImageName', 'Condition', 'Location']).size().reset_index(name='Microglia')
merged_df = pd.merge(grouped_df, image_df[['ImageName','Condition', 'CA1Volume', 'CA3Volume']], on='ImageName')
scale = [0.9278, 0.3459, 0.3459]
# grouped_df.reset_index(inplace=True)
merged_df['MicrogliaHC'] = (merged_df['Microglia'] / merged_df['CA1Volume']) * np.prod(scale) *10**6
merged_df['MicrogliaEDGE'] = (merged_df['Microglia'] / merged_df['CA3Volume']) * np.prod(scale) *10**6
merged_df = merged_df[merged_df['Location'] != 'Undefined']
merged_df['MicrogliaDensity'] = np.where(merged_df['Location'] == 'HC', merged_df['MicrogliaHC'], merged_df['MicrogliaEDGE'])
print(merged_df)

In [ ]:

order = ['Sham', 'Contra', 'Ipsi']
#plt.figure(figsize=(10, 6))  # Adjust size as needed
sns.boxplot(data=merged_df, x='Condition_x', y='MicrogliaDensity', showfliers=False, order=order, hue='Location',palette='Set3', dodge=True)
sns.stripplot(data=merged_df, x='Condition_x', y='MicrogliaDensity', order=order, hue='Location', palette=palette, alpha=0.7,dodge=True)
plt.xlabel('Condition')
plt.ylabel('GFAP positive nuclei / 100 \u03bcm^3')
#plt.ylim(0,10)
plt.tight_layout()
#plt.savefig("plots/GFAPPositive.pdf")
plt.show()

In [ ]:
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

print(grouped_df.keys())

anova_results = f_oneway(merged_df[merged_df['Condition'] == 'Contra']['positiveVolume'],
                         merged_df[merged_df['Condition'] == 'Ipsi']['positiveVolume'],
                         merged_df[merged_df['Condition'] == 'Sham']['positiveVolume'])

# Perform Tukey HSD post hoc test
tukey_results = pairwise_tukeyhsd(merged_df['positiveVolume'], merged_df['Condition'])

print("ANOVA Results:")
print("F-statistic:", anova_results.statistic)
print("p-value:", anova_results.pvalue)

print("\nTukey HSD Results:")
print(tukey_results)



In [ ]:
for name in nucleus_df['ImageName'].unique():
    df = nucleus_df[nucleus_df['ImageName'] == name]
    sns.violinplot(data=df, x='ImageName', y='CytoCh2Intensity')
    plt.show()